In [ ]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers
import math
from lib.loader import SingleFileExtractor, FolderExtractor
import joblib

# ==== Parametry ====
HDF_PATH = "/data"
SIGNAL_NAME = "art"
WINDOW_SIZE = 500
BATCH_SIZE = 64
LATENT_DIM = 100
EPOCHS = 100
MODEL_DIR_SIGNAL_GAN = "models_signal_gan"
os.makedirs(MODEL_DIR_SIGNAL_GAN, exist_ok=True)

# Techniky pro stabilizaci
LABEL_SMOOTHING_REAL = 0.9 
LABEL_SMOOTHING_FAKE = 0.1
LEARNING_RATE_G = 2e-4 
LEARNING_RATE_D = 1e-4

# Pro reprodukovatelnost
tf.random.set_seed(42)
np.random.seed(42)

# ==== Pomocné Funkce pro Načítání Signálů ====
def get_file_paths(folder_path):
    try:
        folder_extractor = FolderExtractor(folder_path)
        return [e._hdf5_file_path for e in folder_extractor._extractors]
    except ValueError as e:
        print(f"Chyba při inicializaci FolderExtractor: {e}. Ujistěte se, že '{folder_path}' je složka.")
        return []
    except Exception as e:
        print(f"Neznámá chyba při získávání cest k souborům: {e}")
        return []

def load_signal_segments(file_path, annotations_folder_path, signal_name="art"):
    extractor = SingleFileExtractor(file_path)
    extractor.auto_annotate(annotations_folder_path)
    good_segments, _ = extractor.extract(signal_name)
    if not good_segments:
        return np.array([])
    extractor.load_data(good_segments)
    concatenated_data = np.concatenate([seg.data for seg in good_segments if seg.data is not None and len(seg.data) > 0])
    return concatenated_data

def signal_to_windows(signal, window_size, step_size=None):
    windows = []
    if step_size is None:
        step_size = window_size // 4
    if len(signal) < window_size:
        return np.array(windows)

    for i in range(0, len(signal) - window_size + 1, step_size):
        window = signal[i:i + window_size]
        if not np.isnan(window).any():
            windows.append(window)
    return np.array(windows)


# ==== Načtení a Zpracování Dat ====
all_files = get_file_paths(HDF_PATH)
all_signal_data_list = []

print("Zpracovávám soubory a načítám segmenty signálu 'art':")
if not all_files:
    print(f"Ve složce '{HDF_PATH}' nebyly nalezeny žádné HDF5 soubory.")
else:
    for file_path in all_files:
        base_name = os.path.basename(file_path)
        print(f" - {base_name}")
        try:
            signal_data = load_signal_segments(file_path, HDF_PATH, SIGNAL_NAME)
            if signal_data.size > WINDOW_SIZE :
                 all_signal_data_list.append(signal_data)
            elif signal_data.size > 0:
                print(f"   Segment z {base_name} je příliš krátký ({signal_data.size}) pro window_size {WINDOW_SIZE}, bude přeskočen.")
        except Exception as e:
            print(f"   Chyba při zpracování souboru {base_name}: {e}")


if not all_signal_data_list:
    print("Nebyly nalezeny žádné validní segmenty signálu pro trénování. Program bude ukončen.")
    X_signal_windows_scaled = np.array([])
    signal_scaler = MinMaxScaler(feature_range=(-1, 1))
else:
    concatenated_total_signal = np.concatenate(all_signal_data_list)
    print(f"Celková délka spojeného signálu: {len(concatenated_total_signal)}")

    if len(concatenated_total_signal) < WINDOW_SIZE:
        print(f"Celková délka spojeného signálu ({len(concatenated_total_signal)}) je menší než WINDOW_SIZE ({WINDOW_SIZE}). Nelze pokračovat.")
        X_signal_windows_scaled = np.array([])
        signal_scaler = MinMaxScaler(feature_range=(-1, 1))
    else:
        signal_scaler = MinMaxScaler(feature_range=(-1, 1))
        scaled_total_signal = signal_scaler.fit_transform(concatenated_total_signal.reshape(-1, 1)).flatten()
        joblib.dump(signal_scaler, os.path.join(MODEL_DIR_SIGNAL_GAN, 'signal_scaler.gz'))

        X_signal_windows = signal_to_windows(scaled_total_signal, WINDOW_SIZE)

        if X_signal_windows.shape[0] == 0:
            print(f"Po vytvoření oken nezůstala žádná data. Zkontrolujte WINDOW_SIZE, délku signálů a step_size.")
            X_signal_windows_scaled = np.array([])
        else:
            print(f"Počet získaných signálových oken: {X_signal_windows.shape[0]}, délka okna: {X_signal_windows.shape[1]}")
            X_signal_windows_scaled = X_signal_windows[:, :, np.newaxis].astype(np.float32)


if X_signal_windows_scaled.size == 0 :
    print("Žádná data pro trénování GANu. Přeskakuji trénování a generování.")
else:
    train_dataset_signal = tf.data.Dataset.from_tensor_slices(X_signal_windows_scaled).shuffle(X_signal_windows_scaled.shape[0]).batch(BATCH_SIZE)


# ==== Architektura GANu pro signály ====
def build_generator_signal(latent_dim, output_window_size):
  model = tf.keras.Sequential(name="Generator_Signal")
  model.add(layers.Input(shape=(latent_dim,)))
  initial_reshape_len = math.ceil(output_window_size / 8.0)
  num_filters_initial_reshape = 128
  initial_dense_units = int(initial_reshape_len * num_filters_initial_reshape)
  model.add(layers.Dense(initial_dense_units))
  model.add(layers.Reshape((int(initial_reshape_len), num_filters_initial_reshape)))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(negative_slope=0.2))
  model.add(layers.Conv1DTranspose(128, kernel_size=5, strides=2, padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(negative_slope=0.2))
  model.add(layers.Conv1DTranspose(64, kernel_size=5, strides=2, padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(negative_slope=0.2))
  model.add(layers.Conv1DTranspose(32, kernel_size=5, strides=2, padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(negative_slope=0.2))
  model.add(layers.Conv1DTranspose(1, kernel_size=5, strides=1, padding='same', activation='tanh'))
  current_length_after_upsample = int(initial_reshape_len * 8)
  if current_length_after_upsample != output_window_size:
      diff = current_length_after_upsample - output_window_size
      if diff < 0:
          raise ValueError(f"Aktuální délka ({current_length_after_upsample}) je menší než cílová ({output_window_size}).")
      crop_start = diff // 2
      crop_end = diff - crop_start
      model.add(layers.Cropping1D(cropping=(crop_start, crop_end)))
  assert model.output_shape == (None, output_window_size, 1), f"Chybný výstupní tvar generátoru: {model.output_shape}"
  return model

def build_discriminator_signal(input_window_size):
    model = tf.keras.Sequential(name="Discriminator_Signal")
    model.add(layers.Input(shape=(input_window_size, 1)))
    model.add(layers.Conv1D(64, kernel_size=5, strides=2, padding='same'))
    model.add(layers.LeakyReLU(negative_slope=0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(128, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(negative_slope=0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(256, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(negative_slope=0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model



# ==== Trénování GANu ====
cross_entropy_signal = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss_signal(real_output, fake_output):
    real_labels = tf.ones_like(real_output) * LABEL_SMOOTHING_REAL
    real_loss = cross_entropy_signal(real_labels, real_output)

    fake_labels = tf.ones_like(fake_output) * LABEL_SMOOTHING_FAKE
    fake_loss = cross_entropy_signal(fake_labels, fake_output)

    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss_signal(fake_output):
  return cross_entropy_signal(tf.ones_like(fake_output), fake_output)

print(f"Používám Label Smoothing pro reálné štítky: {LABEL_SMOOTHING_REAL}")
print(f"Rychlost učení Generátoru: {LEARNING_RATE_G}, Diskriminátoru: {LEARNING_RATE_D}")

generator_optimizer_signal = optimizers.Adam(LEARNING_RATE_G, beta_1=0.5)
discriminator_optimizer_signal = optimizers.Adam(LEARNING_RATE_D, beta_1=0.5)

generator_signal_model = build_generator_signal(LATENT_DIM, WINDOW_SIZE)
discriminator_signal_model = build_discriminator_signal(WINDOW_SIZE)

print("--- Architektura Generátoru ---")
generator_signal_model.summary()
print("\n--- Architektura Diskriminátoru ---")
discriminator_signal_model.summary()

@tf.function
def train_step_gan_signal(signal_windows_batch):
    noise = tf.random.normal([tf.shape(signal_windows_batch)[0], LATENT_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_signals = generator_signal_model(noise, training=True)

        real_output = discriminator_signal_model(signal_windows_batch, training=True)
        fake_output = discriminator_signal_model(generated_signals, training=True)

        gen_loss = generator_loss_signal(fake_output)
        disc_loss = discriminator_loss_signal(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator_signal_model.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator_signal_model.trainable_variables)

    generator_optimizer_signal.apply_gradients(zip(gradients_of_generator, generator_signal_model.trainable_variables))
    discriminator_optimizer_signal.apply_gradients(zip(gradients_of_discriminator, discriminator_signal_model.trainable_variables))

    return gen_loss, disc_loss

def train_gan_signal(dataset, epochs_count):
    print(f"Vstupuji do funkce train_gan_signal, počet epoch: {epochs_count}")
    history_gen_loss = []
    history_disc_loss = []

    for epoch in range(epochs_count):
        print(f"Začátek epochy {epoch+1}/{epochs_count}.")
        epoch_gen_loss_avg = tf.keras.metrics.Mean()
        epoch_disc_loss_avg = tf.keras.metrics.Mean()

        batch_idx = 0
        for signal_batch in dataset:
            if batch_idx == 0 and epoch == 0:
                print(f"   Zpracovávám první dávku ({signal_batch.shape}) v první epoše...")
                print("      Volám train_step_gan_signal poprvé...")

            gen_loss, disc_loss = train_step_gan_signal(signal_batch)

            if batch_idx == 0 and epoch == 0:
                print(f"      První volání train_step_gan_signal dokončeno. G_loss: {gen_loss:.4f}, D_loss: {disc_loss:.4f}")

            epoch_gen_loss_avg.update_state(gen_loss)
            epoch_disc_loss_avg.update_state(disc_loss)
            batch_idx += 1

        g_loss_val = epoch_gen_loss_avg.result().numpy()
        d_loss_val = epoch_disc_loss_avg.result().numpy()
        history_gen_loss.append(g_loss_val)
        history_disc_loss.append(d_loss_val)

        print(f"Epocha {epoch+1}/{epochs_count} dokončena. Ztráta G: {g_loss_val:.4f}, Ztráta D: {d_loss_val:.4f}")

        if (epoch + 1) % 10 == 0 or epoch == epochs_count - 1 :
            generator_signal_model.save(os.path.join(MODEL_DIR_SIGNAL_GAN, f"gan_generator_signal_epoch_{epoch+1}.keras"))

    return history_gen_loss, history_disc_loss

print("\nSpouštím trénování GANu pro signály arteriálního tlaku...")
gen_loss_history_signal, disc_loss_history_signal = train_gan_signal(train_dataset_signal, EPOCHS)


print("Trénování GANu pro signály dokončeno a modely uloženy.")


 # ==== Generování Signálů ====
def generate_signals(generator_model, scaler_model, n_signals=5, latent_dim=LATENT_DIM):
    noise = tf.random.normal([n_signals, latent_dim])
    generated_scaled_signals = generator_model.predict(noise, verbose=0)

    generated_signals_unscaled_list = []
    for i in range(n_signals):
        signal_scaled_one = generated_scaled_signals[i, :, 0]
        signal_unscaled_one = scaler_model.inverse_transform(signal_scaled_one.reshape(-1, 1)).flatten()
        generated_signals_unscaled_list.append(signal_unscaled_one)

    return generated_signals_unscaled_list

    try:
        loaded_signal_scaler = joblib.load(os.path.join(MODEL_DIR_SIGNAL_GAN, 'signal_scaler.gz'))
    except FileNotFoundError:
        print(f"Chyba: Soubor se scalerem '{os.path.join(MODEL_DIR_SIGNAL_GAN, 'signal_scaler.gz')}' nebyl nalezen. Používám scaler z trénování.")
        loaded_signal_scaler = signal_scaler

    generated_art_signals = generate_signals(generator_signal_model, loaded_signal_scaler, n_signals=3)


    # ==== Vizualizace ====
    if X_signal_windows_scaled.size > 0 and len(generated_art_signals) > 0:
        real_signal_window_scaled_example = next(iter(train_dataset_signal))[0].numpy()
        real_signal_window_unscaled_example = loaded_signal_scaler.inverse_transform(real_signal_window_scaled_example[:,0].reshape(-1,1)).flatten()

        plt.figure(figsize=(18, 6 * ((len(generated_art_signals) + 1) // 2)))
        plt.subplot((len(generated_art_signals) + 1) // 2, 2, 1)
        plt.plot(real_signal_window_unscaled_example)
        plt.title(f"Příklad Reálného Signálu 'art' (okno {WINDOW_SIZE} vzorků)")
        plt.xlabel("Vzorek")
        plt.ylabel("Arteriální tlak")

        for i, gen_signal in enumerate(generated_art_signals):
            plt.subplot((len(generated_art_signals) + 1) // 2, 2, i + 2)
            plt.plot(gen_signal)
            plt.title(f"Generovaný Signál 'art' {i+1} (okno {WINDOW_SIZE} vzorků)")
            plt.xlabel("Vzorek")
            plt.ylabel("Arteriální tlak")

        plt.tight_layout()
        plt.show()

    if 'gen_loss_history_signal' in globals() and 'disc_loss_history_signal' in globals():
        plt.figure(figsize=(10, 5))
        plt.plot(gen_loss_history_signal, label='Ztráta Generátoru (signál)')
        plt.plot(disc_loss_history_signal, label='Ztráta Diskriminátoru (signál)')
        plt.title("Historie Ztrát GANu (signál 'art')")
        plt.xlabel('Epocha')
        plt.ylabel('Ztráta')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print("Historie trénování pro GAN na signálech není k dispozici.")

    output_hdf5_generated_signals_file = os.path.join(MODEL_DIR_SIGNAL_GAN, "generated_art_signals.hdf5")
    with h5py.File(output_hdf5_generated_signals_file, "w") as f_out:
        for i, signal_data in enumerate(generated_art_signals):
            dataset_name = f"art_synthetic_{i}"
            f_out.create_dataset(dataset_name, data=signal_data)
    print(f"Vygenerované signály 'art' uloženy do {output_hdf5_generated_signals_file}")


SyntaxError: invalid syntax. Perhaps you forgot a comma? (952532253.py, line 183)

# Načítání knihoven

*   **os** - práce se souborovým systémem
*   **h5py** - čtení/zápis HDF5 souborů (formát pro ukládání velkých dat)
*   **numpy** - práce s poli a numerickými výpočty
*   **matplotlib.pyplot** - kreslení grafů
*   **sklearn.preprocessing.MinMaxScaler** - škálování dat do zadaného rozsahu
*   **tensorflow** - framework pro strojové učení
*   **tensorflow.kera**s - API pro tvorbu modelů neuronových sítí
*   **math** - základní matematické funkce
*   **joblib** - ukládnání a načítání Python objektů
*   **loader** - vytvořený skript pro načítání signálů ze souborů





In [ ]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers
import math
from lib.loader import SingleFileExtractor, FolderExtractor
import joblib

# Nastavení Parametrů

*   **HDF_PATH** - cesta souboru k signálům
*   **SIGNAL_NAME** - jméno signálu které chceme generovat (art - arteriální tlak)
*   **WIDNOW_SIZE** - délka jednoho segmentu signálu, na kterém bude GAN trénovat
*   **BATCH_SIZE** - kolik window_size se zpracuje v jednom tréninkovém kroku
*   **LATENT_DIM** - velikost vstupního vektoru náhodného šumu pro generátor
*   **EPOCHS** - počet opakování celého tréninkového cyklu nad daty

*   **LABEL_SMOOTHING_REAL** - místo toho aby se reálným vzorkům při tréninku dala hodnota 1.0, použije se 0.9 (label smoothing) což pomáhá stabilizovat trénink
*   **LABEL_SMOOTHING_FAKE** - to stejné ale pro falešné hodnoty, místo 0.0 se použije 0.1
*   **LEARNING_RATE_G** - rychlost učení pro generátor v GANu
*   **LEARNING_RATE_D** - rychlost učení pro diskriminátor, je nižžší protože se diskriminátor často učí rychleji než generátor

In [ ]:
# ==== Parametry ====
HDF_PATH = "data"
SIGNAL_NAME = "art"
WINDOW_SIZE = 500
BATCH_SIZE = 64
LATENT_DIM = 100
EPOCHS = 50
MODEL_DIR_SIGNAL_GAN = "models_signal_gan"
os.makedirs(MODEL_DIR_SIGNAL_GAN, exist_ok=True)

LABEL_SMOOTHING_REAL = 0.9 
LABEL_SMOOTHING_FAKE = 0.1
LEARNING_RATE_G = 2e-4 
LEARNING_RATE_D = 1e-4

# Pro reprodukovatelnost
tf.random.set_seed(42)
np.random.seed(42)

# Pomocné funkce

*   get_file_paths - jako input to má cestu k souboru signálům a pomocí FolderExtractor třídy z loader knihovny to vrací cesty ke všem dostupným signálům

* load_signal_segments - načítá specifické signál segmenty z jednoho HDF5 souboru, používá SingleFileExtractor třídu k zprocesování souboru, autimaticky anotuje data a extractuje (zatím) pouze dobré segmenty, pak je vrací jako jeden numpy array

*   signal_to_windows - tato funkce bere jedno dimensionální signál (numpy array), window_size a step_size jako vstup. funkce iteruje skrze signál, extractuje okna a divá se jestli někde nejsou NaN hodnoty. Pouze okna které nemají NaN hodnoty jsou přidány do seznamu. Nakonec se tento seznam vrací jako numpy array.

In [ ]:
# ==== Pomocné Funkce pro Načítání Signálů ====
def get_file_paths(folder_path):
    try:
        folder_extractor = FolderExtractor(folder_path)
        return [e._hdf5_file_path for e in folder_extractor._extractors]
    except ValueError as e:
        print(f"Chyba při inicializaci FolderExtractor: {e}. Ujistěte se, že '{folder_path}' je složka.")
        return []
    except Exception as e:
        print(f"Neznámá chyba při získávání cest k souborům: {e}")
        return []

def load_signal_segments(file_path, annotations_folder_path, signal_name="art"):
    extractor = SingleFileExtractor(file_path)
    extractor.auto_annotate(annotations_folder_path)
    good_segments, _ = extractor.extract(signal_name)
    if not good_segments:
        return np.array([])
    extractor.load_data(good_segments)
    concatenated_data = np.concatenate([seg.data for seg in good_segments if seg.data is not None and len(seg.data) > 0])
    return concatenated_data

def signal_to_windows(signal, window_size, step_size=None):
    windows = []
    if step_size is None:
        step_size = window_size // 4
    if len(signal) < window_size:
        return np.array(windows)

    for i in range(0, len(signal) - window_size + 1, step_size):
        window = signal[i:i + window_size]
        if not np.isnan(window).any():
            windows.append(window)
    return np.array(windows)


# Načtení a zpracování dat

Tato skece kódu obstarává načitání, proceosvání a přípravu signálu z HDF5 soborů pro použití při trenování GAN.


Jako první se snaží dostat seznam všech cest k souborům signálu ve složce definované v `HDF_PATH` pomocí funkce `get_file_paths` která je výšše

Poté iteruje přes každou cestu k souboru a načte signál specifikovaný v `SIGNAL_NAME` pomocí funkce `load_signal_segments` a přidá načtené signály do `all_signal_data_list` seznamu.

Poté zřetězí všechny nalezené signály do jednoho pole, vypíše jeho délku a podívá se jestli totální délka je dostatečná k vytvoření alespoň jedno okno s velikostí `WINDOW_SIZE`.

Pokud je délka dostatečná tak se inicializuje `MinMaxScaler` který data vyškáluje do rozsahu (-1, 1).

To je poté rozděleno do překryvajících se oken s velikostí `WINDOW_SIZE` pomocí `signal_to_window`funkce výše.

Jestliže okna byla úspešně vytvořena, vypíše to počet oken a jejich velikost, potom je přetvoří window data k přidání extra dimenzi, aby se to mohlo využít jako vstup pro konvoluční vrstvy v GAN modelch.

Nakonec se z signálových oken vytvoří `tf.data.Dataset`. Tento dataset se pak zamíchá pro náhodnost pořadí oken a batchnou do skupin o velikosti `BATCH_SIZE`.

Tento připravený dataset (`train_dataset_signal`) bude použit k následujícímu GAN trénování.


In [ ]:
# ==== Načtení a Zpracování Dat ====
all_files = get_file_paths(HDF_PATH)
all_signal_data_list = []

print("Zpracovávám soubory a načítám segmenty signálu 'art':")
if not all_files:
    print(f"Ve složce '{HDF_PATH}' nebyly nalezeny žádné HDF5 soubory.")
else:
    for file_path in all_files:
        base_name = os.path.basename(file_path)
        print(f" - {base_name}")
        try:
            signal_data = load_signal_segments(file_path, HDF_PATH, SIGNAL_NAME)
            if signal_data.size > WINDOW_SIZE :
                 all_signal_data_list.append(signal_data)
            elif signal_data.size > 0:
                print(f"   Segment z {base_name} je příliš krátký ({signal_data.size}) pro window_size {WINDOW_SIZE}, bude přeskočen.")
        except Exception as e:
            print(f"   Chyba při zpracování souboru {base_name}: {e}")


if not all_signal_data_list:
    print("Nebyly nalezeny žádné validní segmenty signálu pro trénování. Program bude ukončen.")
    X_signal_windows_scaled = np.array([])
    signal_scaler = MinMaxScaler(feature_range=(-1, 1))
else:
    concatenated_total_signal = np.concatenate(all_signal_data_list)
    print(f"Celková délka spojeného signálu: {len(concatenated_total_signal)}")

    if len(concatenated_total_signal) < WINDOW_SIZE:
        print(f"Celková délka spojeného signálu ({len(concatenated_total_signal)}) je menší než WINDOW_SIZE ({WINDOW_SIZE}). Nelze pokračovat.")
        X_signal_windows_scaled = np.array([])
        signal_scaler = MinMaxScaler(feature_range=(-1, 1))
    else:
        signal_scaler = MinMaxScaler(feature_range=(-1, 1))
        scaled_total_signal = signal_scaler.fit_transform(concatenated_total_signal.reshape(-1, 1)).flatten()
        joblib.dump(signal_scaler, os.path.join(MODEL_DIR_SIGNAL_GAN, 'signal_scaler.gz'))

        X_signal_windows = signal_to_windows(scaled_total_signal, WINDOW_SIZE)

        if X_signal_windows.shape[0] == 0:
            print(f"Po vytvoření oken nezůstala žádná data. Zkontrolujte WINDOW_SIZE, délku signálů a step_size.")
            X_signal_windows_scaled = np.array([])
        else:
            print(f"Počet získaných signálových oken: {X_signal_windows.shape[0]}, délka okna: {X_signal_windows.shape[1]}")
            X_signal_windows_scaled = X_signal_windows[:, :, np.newaxis].astype(np.float32)


if X_signal_windows_scaled.size == 0 :
    print("Žádná data pro trénování GANu. Přeskakuji trénování a generování.")
else:
    train_dataset_signal = tf.data.Dataset.from_tensor_slices(X_signal_windows_scaled).shuffle(X_signal_windows_scaled.shape[0]).batch(BATCH_SIZE)


Zpracovávám soubory a načítám segmenty signálu 'art':
 - TBI_021_v2_1_4_18.hdf5
 - TBI_022_v2_1_6_21.hdf5
 - TBI_021_v2_1_4_2.hdf5
 - TBI_022_v2_2_1_15.hdf5
 - TBI_023_v2_2_2_17.hdf5
 - TBI_019_v2_1_6_6.hdf5
 - TBI_024_v2_1_6_5.hdf5
 - TBI_024_v2_1_3_19.hdf5
 - TBI_021_v2_1_3_11.hdf5
 - TBI_024_v2_1_3_0.hdf5
 - TBI_019_v2_1_6_17.hdf5
 - TBI_023_v2_1_6_14.hdf5
 - TBI_019_v2_1_5_10.hdf5
 - TBI_022_v2_1_7_18.hdf5
 - TBI_022_v2_1_7_11.hdf5
 - TBI_022_v2_2_2_6.hdf5
 - TBI_024_v2_1_5_15.hdf5
 - TBI_021_v2_2_5_2.hdf5
 - TBI_021_v2_1_5_15.hdf5
 - TBI_023_v2_2_2_7.hdf5
 - TBI_023_v2_2_1_8.hdf5
 - TBI_023_v2_2_1_6.hdf5
 - TBI_019_v2_1_4_13.hdf5
 - TBI_022_v2_2_2_15.hdf5
 - TBI_023_v2_2_3_3.hdf5
 - TBI_019_v2_1_5_19.hdf5
 - TBI_023_v2_1_6_5.hdf5
 - TBI_023_v2_1_6_1.hdf5
 - TBI_019_v2_1_4_18.hdf5
 - TBI_024_v2_1_3_10.hdf5
 - TBI_021_v2_2_2_17.hdf5
 - TBI_021_v2_1_3_17.hdf5
 - TBI_019_v2_1_5_12.hdf5
 - TBI_019_v2_2_1_0.hdf5
 - TBI_021_v2_1_5_22.hdf5
 - TBI_022_v2_1_4_7.hdf5
 - TBI_016_v2_1_1_12.hdf

# Architektura GANu pro signály

- tato sekce definuje architekturu Generative Adversarial Network (GAN)  specificky definovanou pro generaci signálu. GAN obashuje dva hlavní díly: **Generátor** a **Diskriminátor**


## Generátor (`build_generator_signal`)
 - role generátoru je generovat syntetické data která se podobají reálným datům. Bere náhodný šum vektor jako vstup a transformuje to do signálu.

 - vytváří `tf.keras.Sequential` model, který je linerání stoh vrstev
 -`layers.Input(shape=(latent_dim,))` specifikuje vstupní tvar, který je vektor velikosti `latent_dim`. Tohle je šum vektor.
 - `layers.Dense` a následující `layers.Reshape` vrsva transformuje vstup šum vektoru do tvaru vhodného pro konvoluční vrstvy. `initial_reshape_len` a `num_filters_initial_reshape`jsou spočítaná na základě požadované `output_window_size` hodnotě která zajistí že se signál převzorkuje správně.
 - `layers.BatchNormalization` vrstva pomáhá stabilizovat trénovací proces.
 - `layers.LeakyReLU` je aktivační funkce která zavádí nelinearita
 - `layers.Conv1DTranspose` (také známý jako dekonvoluční nebo převzorkovací vrstvy) jsou použita pro zvýšení délky signálu. Jsou inverzí konvolučních vrstev.
 - Generátor používá několik `layers.Conv1DTranspose` vrstev s krokem 2, což efektivně zdvojnásobuje délku signálu při každém kroku
 - Finální `layers.Conv1DTranspose` vrstva má jediný filter a to `tanh` aktivační funkc, která škáluje výsledné hodnoty do rozsahu -1 až 1, aby to sedělo se škálováním trénovacích dat.
 - `layers.Cropping1D` vrstva je použita pro upravení délky generovaného signálu pokud ten převzorkovací proces nevratí výsledek v `output_window_size` hodnotě.
 - `assert` slouží k zajištění tomu že finální výstupní tvar generátoru bude správný: `(None, output_window_size, 1)`, kde `None` reprezentuje batch size, `output_window_size` je délka vygenerovaného signál okna, a `1` je počet kanálů (pro jeden signál).


## Diskriminátor (`build_discriminator_signal`)
- role diskriminátoru je klasifikovat jestli je daný signál reálný (z tréninkových dat) nebo falešný (vygenerovaný generátorem).

- také používá `tf.keras.Sequential` API.
- `layers.Input(shape=(input_window_size, 1))` specifikuje vstupní tvar, který je okno signálu o velikosti `input_window_size` s jedním kanálem.
- `layers.Conv1D` vrstvy jsou použité k extrakci prvků ze signálu. Tyto vrstvy aplikují konvoluční filtry s časovou dimenzí signálu.
- `strides=2` parametr v `Conv1D` vrstvách snižuje délku signálu v každém kroku.
- `layers.LeakyReLU` aktivace je použitá po konvolučních vrstvách.
- `layers.BatchNormalization` je aplikován pro stabilizaci trénování.
- `layers.Dropout` je zahrnuta jako regularizační technika, aby se zabránilo přeplnění.
- `layers.Flatten` konvertuje 1D konvoluční prvky map do plochého vektoru.
- finální `layers.Dense` má jednu výstupní jednotku a používá `sigmoid` aktivační funkci. To vrátí hodnotu mezi 0 a 1, což představuje pravděpodobnost jestli je signál reálný (blíže k 1) nebo falešný (blíže k 0)

In [ ]:
# ==== Architektura GANu pro signály ====
def build_generator_signal(latent_dim, output_window_size):
  model = tf.keras.Sequential(name="Generator_Signal")
  model.add(layers.Input(shape=(latent_dim,)))
  initial_reshape_len = math.ceil(output_window_size / 8.0)
  num_filters_initial_reshape = 128
  initial_dense_units = int(initial_reshape_len * num_filters_initial_reshape)
  model.add(layers.Dense(initial_dense_units))
  model.add(layers.Reshape((int(initial_reshape_len), num_filters_initial_reshape)))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(negative_slope=0.2))
  model.add(layers.Conv1DTranspose(128, kernel_size=5, strides=2, padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(negative_slope=0.2))
  model.add(layers.Conv1DTranspose(64, kernel_size=5, strides=2, padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(negative_slope=0.2))
  model.add(layers.Conv1DTranspose(32, kernel_size=5, strides=2, padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(negative_slope=0.2))
  model.add(layers.Conv1DTranspose(1, kernel_size=5, strides=1, padding='same', activation='tanh'))
  current_length_after_upsample = int(initial_reshape_len * 8)
  if current_length_after_upsample != output_window_size:
      diff = current_length_after_upsample - output_window_size
      if diff < 0:
          raise ValueError(f"Aktuální délka ({current_length_after_upsample}) je menší než cílová ({output_window_size}).")
      crop_start = diff // 2
      crop_end = diff - crop_start
      model.add(layers.Cropping1D(cropping=(crop_start, crop_end)))
  assert model.output_shape == (None, output_window_size, 1), f"Chybný výstupní tvar generátoru: {model.output_shape}"
  return model

def build_discriminator_signal(input_window_size):
    model = tf.keras.Sequential(name="Discriminator_Signal")
    model.add(layers.Input(shape=(input_window_size, 1)))
    model.add(layers.Conv1D(64, kernel_size=5, strides=2, padding='same'))
    model.add(layers.LeakyReLU(negative_slope=0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(128, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(negative_slope=0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(256, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(negative_slope=0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model


# Trénování GANu

tato sekce kódu se zaměřuje na definování a provedení trénovacího procesu pro GAN

## Definice Ztrátových funkcí

*  `cross_entropy_signal`: inicializuje binární křížovou entropii, což je běžná ztrátová funkce pro klasifikační úlohy, jako je ta, kterou řeší diskriminátor (rozhodování mezi reálným a falešným)

*    `discriminator_loss_signal(real_output, fake_output)`: Tato funkce počítá ztrátu pro **diskriminátor**.
  *   Bere `real_ouput`(výstup diskriminátoru pro reálné signály) a `fake_output` (výstup diskriminátoru pro falešné signály)
  * Pro **reálné signály** používá **label smoothing**. Místo tvrdého štítku `1.0` (reálný), používá `LABEL_SMOOTHING_REAL` (nastaveno na 0.9). Tím se diskriminátor povzbudí k méně jistým předpovědím pro reálná data, což může pomoci stabilizovat trénink.
  * Pro **falešné signály** také používá **label smoothing** s `LABEL_SMOOTHING_FAKE` (nastaven na 0.1).
  * Totální ztráta diskriminátoru je součet `real_loss` a `fake_loss`. Tento diskriminátor se snaží tyto ztráty minimalizovat.

*   `generator_loss_signal(fake_output)`: Tato funkce počítá ztrátu pro **generátor**.
  * Bere `fake_output` (výstup diskriminátoru pro falešné signály vygenerované generátorem).
  * Cílem generátoru je oklamat diskriminátor, tj. aby diskriminátor ohodnotil falešné signály za "reálné". Proto se generátor snaží maximalizovat ztrátu generátoru na falešných datech, což je ekvivalentní minimalizaci **své vlastní ztráty** tím, že se pokouší přimět diskriminátor k předpovědi `1.0` pro falešná data

## Optimalizátory a Modely

*   Kód vypisuje použité hodnoty pro label a rychlosti učení pro přehlednost.
*   `generator_optimizer_signal` a `discriminator_optimizer_signal`: Inicializují **Adam optimalizátory** pro generátor a diskriminátor. Každý má svou vlastní rychlost učení (`LEARNING_RATE_G` a `LEARNING_RATE_D`). Diskriminátor má obvykle nižžší rychlost učení, protože se učí rychleji než generátor a je důležité udržet rovnováhu v tréninku. Parametr `beta_1=0.5` je běžné nastavené pro GAN trénink.
*   `generator_signal_model` a `discriminator_signal_model`: Inicializují instance modelů generátorů a diskriminátorů pomocí dříve definovaných funkcí `build_generator_signal` a `build_discriminator_signal`. Předávají se jim odpovídající parametry (`LATENT_DIM` pro generátor, `WINDOW_SIZE` pro oba).

## Trénovací Krok (`train_step_gan_signal`)
*   `tf.function`: Tato dekorace optimalizuje funkci `train_step_gan_signal` pro rychlejší spuštění pomocí TensorFlow grafů.
* Tato funkce provádí **jeden trénovací krok** pro GAN (jak pro generátor, tak pro diskriminátor) na jedné dávce dat.
* `nois = tf.random.normal(...)`: Generuje **náhodný šum** pro vstup generátoru. Velikost šumu odpovíá počtu signálů v aktuální dávce (`tf.shape(signal_widnows_batch) [0]`) a `LATENT_DIM`.
* `with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:` - Toto vytváří **Gradient Tape**. TensorFlow používá Gradient Tapes k zaznamenávání operací, které se mají provést, aby se později mohly automaticky vypočítat gradienty(derivace), které jsou nezbytné pro aktualizaci vah modelu během tréninku. `gen_tape` zaznamenává operace pro generátor a `disc_tape` pro diskriminátor.
* `generated_signals = generator_signal_model(noise, training=True)`: Generátor přijímá náhodný šum a vytváří **falešné signály**. `training=True` zajišťuje, že se během tohoto kroku správně chovají vrstvy jako Batch Normalization a Dropout.
* `real_output = discriminator_signal_model(signal_widnows_batch, training=True)`: Diskriminátor přijímá **reálné signály** z aktuální dávky trénovacích dat a předpovídá, jak "reálné" je považuje.
* `fake_output = discriminator_signal_model(generated_signals, training=True)`: Diskriminátor přijímá **falešné signály** vygenerované generátorem a předpovídá, jak "reálné" je považuje.
* `gen_loss = generator_loss_signal(fake_output)`: Vypočítá **ztrátu generátoru** na základě výstupu diskriminátoru pro falešné signály.
* `disc_loss = discriminator_loss_signal(real_output, fake_output)`: Vypočítá **ztrátu diskriminátoru** na základě výstupu pro reálné i falešné signály.
* `gradients_of_generator = gen_tape.gradient(gen_loss, generator_signal_model.trainable_varialbes)`: Vypočítá **gradienty ztráty generátoru** vzhledem k trénovatelným proměnným (vahám) generátoru.
* `gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator_signal_model.trainable_variables)`: Vypočítá **gradienty ztráty diskriminátoru** vzhledem k trénovatelným proměnným diskriminátoru.
* `generator_optimizer_signal.apply_gradients(...)` a `discriminator_optimizer_signal.apply_gradients(...)` Tyto řádky **aktualizují váhy generátoru a diskriminátoru pomocí vypočítaných gradientů a jejich příslušných optimalizátorů. Tím se modely učí minimalizovat své ztráty. Funkce vrací vypočítané ztráty generátoru a diskriminátoru pro tuto dávku.

## Hlavní Trénovací Smyčka (`train_gan_signal`)

* Tato funkce řídí celý proces trénovaní GANu pro zadaný počet epoch
* Jako vstup bere `dataset`(připravený `tf.data.Dataset` s trénovacími daty) a `epochs_count` (počet trénovacích epoch)
* Inicialzují se prázdné seznamy pro ukládání historie ztrát generátoru a diskriminátoru v průběhu epoch.
* Hlavní smyčka iteruje přes zadaný počet epoch.
* Uvnitř smyčky pro každou epochu se inicializují instace tf.keras.metrics.Mean() pro sledování **průměrné ztráty generátoru a diskriminátoru** za aktuální epochu.
* Vnitřní smyčka iteruje přes **dávky dat** v `datasetu`.
* Pro každou dávku dat se zavolá funkce `train_step_gan_signal` k provedení jednoho trénovacího kroku a získání ztrát pro tuto dávku.
* Ztráty z aktuální dávky se aktualizují v metríkách průměrných za epochu.
* Po zpracování všech dávek v epoše se získá **finální průměrná ztráta** pro generátor a diskrimátor za tuto epochu.
* Průměrné ztráty za epochu se přidájí do seznamů historie ztrát
* Každých 10 (nebo v poslední epoše) se **uloží model generátoru**. To umožnuje obnovit trénink nebo použít generátor z konkrétní fáze tréninku. Model se ukládá ve formátu `.keras`
* Po dokončení všech epoch funkce vrací seznamy historie ztrát generátoru a diskriminátoru.

## Spuštění Tréninku
* Volá se funkce `train_gan_signal`, která spouští trénovací smyšku s připraveným datasetem (train_dataset_signal) a zadaným počtem epoch (EPOCHS). Výsledné historie ztrát se ukládají do proměnných `gen_loss_history_signal` a `disc_loss_history_signal`.

In [ ]:
# ==== Trénování GANu ====
cross_entropy_signal = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss_signal(real_output, fake_output):
    real_labels = tf.ones_like(real_output) * LABEL_SMOOTHING_REAL
    real_loss = cross_entropy_signal(real_labels, real_output)

    fake_labels = tf.ones_like(fake_output) * LABEL_SMOOTHING_FAKE
    fake_loss = cross_entropy_signal(fake_labels, fake_output)

    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss_signal(fake_output):
  return cross_entropy_signal(tf.ones_like(fake_output), fake_output)

print(f"Používám Label Smoothing pro reálné štítky: {LABEL_SMOOTHING_REAL}")
print(f"Rychlost učení Generátoru: {LEARNING_RATE_G}, Diskriminátoru: {LEARNING_RATE_D}")

generator_optimizer_signal = optimizers.Adam(LEARNING_RATE_G, beta_1=0.5)
discriminator_optimizer_signal = optimizers.Adam(LEARNING_RATE_D, beta_1=0.5)

generator_signal_model = build_generator_signal(LATENT_DIM, WINDOW_SIZE)
discriminator_signal_model = build_discriminator_signal(WINDOW_SIZE)

print("--- Architektura Generátoru ---")
generator_signal_model.summary()
print("\n--- Architektura Diskriminátoru ---")
discriminator_signal_model.summary()

@tf.function
def train_step_gan_signal(signal_windows_batch):
    noise = tf.random.normal([tf.shape(signal_windows_batch)[0], LATENT_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_signals = generator_signal_model(noise, training=True)

        real_output = discriminator_signal_model(signal_windows_batch, training=True)
        fake_output = discriminator_signal_model(generated_signals, training=True)

        gen_loss = generator_loss_signal(fake_output)
        disc_loss = discriminator_loss_signal(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator_signal_model.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator_signal_model.trainable_variables)

    generator_optimizer_signal.apply_gradients(zip(gradients_of_generator, generator_signal_model.trainable_variables))
    discriminator_optimizer_signal.apply_gradients(zip(gradients_of_discriminator, discriminator_signal_model.trainable_variables))

    return gen_loss, disc_loss

def train_gan_signal(dataset, epochs_count):
    print(f"Vstupuji do funkce train_gan_signal, počet epoch: {epochs_count}")
    history_gen_loss = []
    history_disc_loss = []

    for epoch in range(epochs_count):
        print(f"Začátek epochy {epoch+1}/{epochs_count}.")
        epoch_gen_loss_avg = tf.keras.metrics.Mean()
        epoch_disc_loss_avg = tf.keras.metrics.Mean()

        batch_idx = 0
        for signal_batch in dataset:
            if batch_idx == 0 and epoch == 0:
                print(f"   Zpracovávám první dávku ({signal_batch.shape}) v první epoše...")
                print("      Volám train_step_gan_signal poprvé...")

            gen_loss, disc_loss = train_step_gan_signal(signal_batch)

            if batch_idx == 0 and epoch == 0:
                print(f"      První volání train_step_gan_signal dokončeno. G_loss: {gen_loss:.4f}, D_loss: {disc_loss:.4f}")

            epoch_gen_loss_avg.update_state(gen_loss)
            epoch_disc_loss_avg.update_state(disc_loss)
            batch_idx += 1

        g_loss_val = epoch_gen_loss_avg.result().numpy()
        d_loss_val = epoch_disc_loss_avg.result().numpy()
        history_gen_loss.append(g_loss_val)
        history_disc_loss.append(d_loss_val)

        print(f"Epocha {epoch+1}/{epochs_count} dokončena. Ztráta G: {g_loss_val:.4f}, Ztráta D: {d_loss_val:.4f}")

        if (epoch + 1) % 10 == 0 or epoch == epochs_count - 1 :
            generator_signal_model.save(os.path.join(MODEL_DIR_SIGNAL_GAN, f"gan_generator_signal_epoch_{epoch+1}.keras"))

    return history_gen_loss, history_disc_loss

print("\nSpouštím trénování GANu pro signály arteriálního tlaku...")
gen_loss_history_signal, disc_loss_history_signal = train_gan_signal(train_dataset_signal, EPOCHS)


print("Trénování GANu pro signály dokončeno a modely uloženy.")


Používám Label Smoothing pro reálné štítky: 0.9
Rychlost učení Generátoru: 0.0002, Diskriminátoru: 0.0001
--- Architektura Generátoru ---


Model: "Generator_Signal"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8064)           │       814,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 63, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 63, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 63, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose                │ (None, 126, 128)       │        82,048 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 126, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 126, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_1              │ (None, 252, 64)        │        41,024 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 252, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 252, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_2              │ (None, 504, 32)        │        10,272 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 504, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 504, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_3              │ (None, 504, 1)         │           161 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cropping1d (Cropping1D)         │ (None, 500, 1)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 949,377 (3.62 MB)

 Trainable params: 948,673 (3.62 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 250, 64)        │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 250, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 250, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 125, 128)       │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 125, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 125, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 125, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 63, 256)        │       164,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 63, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ (None, 63, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 63, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16128)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │        16,129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 223,233 (872.00 KB)

 Trainable params: 222,465 (869.00 KB)

 Non-trainable params: 768 (3.00 KB)


Spouštím trénování GANu pro signály arteriálního tlaku...
Vstupuji do funkce train_gan_signal, počet epoch: 100
Začátek epochy 1/100.
   Zpracovávám první dávku ((64, 500, 1)) v první epoše...
      Volám train_step_gan_signal poprvé...
      První volání train_step_gan_signal dokončeno. G_loss: 0.8994, D_loss: 1.7559
Epocha 1/100 dokončena. Ztráta G: 3.7225, Ztráta D: 0.6177
Začátek epochy 2/100.
Epocha 2/100 dokončena. Ztráta G: 5.5056, Ztráta D: 0.4115
Začátek epochy 3/100.
Epocha 3/100 dokončena. Ztráta G: 5.2955, Ztráta D: 0.4140
Začátek epochy 4/100.
Epocha 4/100 dokončena. Ztráta G: 4.5746, Ztráta D: 0.5074
Začátek epochy 5/100.
Epocha 5/100 dokončena. Ztráta G: 5.2065, Ztráta D: 0.4707
Začátek epochy 6/100.
Epocha 6/100 dokončena. Ztráta G: 5.9587, Ztráta D: 0.3727
Začátek epochy 7/100.
Epocha 7/100 dokončena. Ztráta G: 5.3986, Ztráta D: 0.4314
Začátek epochy 8/100.
Epocha 8/100 dokončena. Ztráta G: 4.7288, Ztráta D: 0.5356
Začátek epochy 9/100.
Epocha 9/100 dokončena. Ztráta 

# Generace signálu a vizualizace

Tato sekce kodu se zaměřuje na generování syntetických signálů použitím natrénovaného GAN generátoru.

*   `generate_signals(generator_model, scaler_model, n_signals=, latent_dim=LATENT_DIM)`
  * Bere natrénovaný `generator_model`, ke škálování dat se použije `scaler_model`, `n_signals` je požadovaný počet signálu k vygenerování, a `latent_dim`(velikost noise vektoru) jako vstup
  * `noise = tf.random.normal([n_signals, latent_dim])`: Tento řádek generuje batch náhodného noise vektoru. Velikost batch je `n_signals` a každý vektor má délku `latent_dim`
  *  `generated_scaled_signals = generator_model.predict(noise, verbose=0)`: Vygenerovaný šum je nakrmen do `generator_model`. Ta `predict` metoda je použita protože používáme ten generátor který už je naučený, `verbose=0` potlačuje výstup z toho prediktivního procesu. Výstup `generated_scaled_signals` je batch syntetických signálů která jsou stále v škálovaném rozsahu (-1 až 1)
  * Kod pak iteruje skrze všechny `n_signals` vygenerovaných signálů.
  * `signal_unscaled_one = scaler_mode.inverse_transform(signal_scaled_one.reshape(-1, 1)).flatten()`: Toto je důležitý krok. Vyškálované signály se musí vrátit do původníhu rozsahu a to pomocí `inverse_transform` metody z `scaler_model`. `reshape(-1,1)` říká jaký tvar má ted. `flatten()` konvertuje výsledný 2D pole zpátky do 1D pole což reprezentuje neškálovaný signál.
  * Tato funkce pak vrací seznam neškálovaných syntetických signálů.

*   Po definici funkce se snažíme načíst uložený `MinMaxScaler` objekt použitím `joblib.load`. Toto je důležité když chceme použít generační proces v separátním skriptu po trénování.

* `generated_art_signals = generate_signals(generator_signal_model, loaded_signal_scaler, n_signals=3)`: Tento řádek volá `generate_signals` funkci k vytvoření 3 syntetických "art" signálů s použitím natrénovaného `generator_signal_model` a načteného `loaded_signal_scaler`

* Poté jsou vykreslena pomocí `plt` a uložena jako hdf5 soubory



In [ ]:
 # ==== Generování Signálů ====
def generate_signals(generator_model, scaler_model, n_signals=5, latent_dim=LATENT_DIM):
    noise = tf.random.normal([n_signals, latent_dim])
    generated_scaled_signals = generator_model.predict(noise, verbose=0)

    generated_signals_unscaled_list = []
    for i in range(n_signals):
        signal_scaled_one = generated_scaled_signals[i, :, 0]
        signal_unscaled_one = scaler_model.inverse_transform(signal_scaled_one.reshape(-1, 1)).flatten()
        generated_signals_unscaled_list.append(signal_unscaled_one)

    return generated_signals_unscaled_list

    try:
        loaded_signal_scaler = joblib.load(os.path.join(MODEL_DIR_SIGNAL_GAN, 'signal_scaler.gz'))
    except FileNotFoundError:
        print(f"Chyba: Soubor se scalerem '{os.path.join(MODEL_DIR_SIGNAL_GAN, 'signal_scaler.gz')}' nebyl nalezen. Používám scaler z trénování.")
        loaded_signal_scaler = signal_scaler

    generated_art_signals = generate_signals(generator_signal_model, loaded_signal_scaler, n_signals=3)


    # ==== Vizualizace ====
    if X_signal_windows_scaled.size > 0 and len(generated_art_signals) > 0:
        real_signal_window_scaled_example = next(iter(train_dataset_signal))[0].numpy()
        real_signal_window_unscaled_example = loaded_signal_scaler.inverse_transform(real_signal_window_scaled_example[:,0].reshape(-1,1)).flatten()

        plt.figure(figsize=(18, 6 * ((len(generated_art_signals) + 1) // 2)))
        plt.subplot((len(generated_art_signals) + 1) // 2, 2, 1)
        plt.plot(real_signal_window_unscaled_example)
        plt.title(f"Příklad Reálného Signálu 'art' (okno {WINDOW_SIZE} vzorků)")
        plt.xlabel("Vzorek")
        plt.ylabel("Arteriální tlak")

        for i, gen_signal in enumerate(generated_art_signals):
            plt.subplot((len(generated_art_signals) + 1) // 2, 2, i + 2)
            plt.plot(gen_signal)
            plt.title(f"Generovaný Signál 'art' {i+1} (okno {WINDOW_SIZE} vzorků)")
            plt.xlabel("Vzorek")
            plt.ylabel("Arteriální tlak")

        plt.tight_layout()
        plt.show()

    if 'gen_loss_history_signal' in globals() and 'disc_loss_history_signal' in globals():
        plt.figure(figsize=(10, 5))
        plt.plot(gen_loss_history_signal, label='Ztráta Generátoru (signál)')
        plt.plot(disc_loss_history_signal, label='Ztráta Diskriminátoru (signál)')
        plt.title("Historie Ztrát GANu (signál 'art')")
        plt.xlabel('Epocha')
        plt.ylabel('Ztráta')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print("Historie trénování pro GAN na signálech není k dispozici.")

    output_hdf5_generated_signals_file = os.path.join(MODEL_DIR_SIGNAL_GAN, "generated_art_signals.hdf5")
    with h5py.File(output_hdf5_generated_signals_file, "w") as f_out:
        for i, signal_data in enumerate(generated_art_signals):
            dataset_name = f"art_synthetic_{i}"
            f_out.create_dataset(dataset_name, data=signal_data)
    print(f"Vygenerované signály 'art' uloženy do {output_hdf5_generated_signals_file}")